In [9]:
import os
os.environ["TRANSFORMERS_NO_TF"] = "1"


In [10]:
#import sklearn as sk
import transformers
#import sentence-transformers
from  datasets import Dataset 
import torch 
import pandas as pd
from transformers import Trainer, TrainingArguments

#from tensorflow import keras

In [11]:



df = pd.read_csv("/home/profniggastein/PycharmProjects/ReducedDataset/gcj2009_balanced_exp.csv")

df = df[["flines", "solution"]].dropna()
df = df.rename(columns={"flines": "code", "solution": "label"})
df["label"] = df["label"].astype(int)


# df = df[["code", "Correct/incorrect"]].dropna()
# df = df.rename(columns={"Correct/incorrect": "label"})
# df["label"] = df["label"].astype(int)


dataset = Dataset.from_pandas(df)


In [12]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from datasets import Value

tokenizer = AutoTokenizer.from_pretrained("Salesforce/codet5-base")


def preprocess(examples):
    return tokenizer(examples["code"], truncation=True, padding="max_length", max_length=256)

tokenized_dataset = dataset.map(preprocess, batched=True)
tokenized_dataset = tokenized_dataset.train_test_split(test_size=0.2)
train_dataset = tokenized_dataset["train"]
eval_dataset = tokenized_dataset["test"]
train_dataset.set_format("torch", columns=["input_ids", "attention_mask", "label"])
eval_dataset.set_format("torch", columns=["input_ids", "attention_mask", "label"])


/home/profniggastein/PycharmProjects/ReducedDataset/.venv/lib/python3.11/site-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Map: 100%|██████████| 3000/3000 [00:00<00:00, 4539.44 examples/s]


In [13]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("Salesforce/codet5-base", num_labels=2)



/home/profniggastein/PycharmProjects/ReducedDataset/.venv/lib/python3.11/site-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of T5ForSequenceClassification were not initialized from the model checkpoint at Salesforce/codet5-base and are newly initialized: ['classification_head.dense.bias', 'classification_head.dense.weight', 'classification_head.out_proj.bias', 'classification_head.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [14]:
training_args = TrainingArguments(
    output_dir="./codeT5-correctness-gcj(Bin-exp2)",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    learning_rate= 2e-5,
    num_train_epochs=10,
    #evaluation_strategy="epoch",
    save_strategy="epoch",
    logging_dir="./logs",
    logging_steps=10,
)

In [22]:
from sklearn.metrics import accuracy_score, f1_score
def compute_metrics(eval_pred):
    logits = eval_pred.predictions
    labels = eval_pred.label_ids
    preds = torch.argmax(torch.tensor(logits), dim=2).numpy()
    return {
        "accuracy": accuracy_score(labels, preds),
        "f1": f1_score(labels, preds)
    }


In [16]:




trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics
)

trainer.train()


/home/profniggastein/PycharmProjects/ReducedDataset/.venv/lib/python3.11/site-packages/torch/utils/data/dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Step,Training Loss
10,0.895300
20,0.804100
30,0.834800
40,0.701700
50,0.815900
60,0.755600
70,0.788100
80,0.824400
90,0.719000
100,0.769700


/home/profniggastein/PycharmProjects/ReducedDataset/.venv/lib/python3.11/site-packages/torch/utils/data/dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/home/profniggastein/PycharmProjects/ReducedDataset/.venv/lib/python3.11/site-packages/torch/utils/data/dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/home/profniggastein/PycharmProjects/ReducedDataset/.venv/lib/python3.11/site-packages/torch/utils/data/dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)
/home/profniggastein/PycharmProjects/ReducedDataset/.venv/lib/python3.11/site-packages/torch/utils/data/dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator i

TrainOutput(global_step=3000, training_loss=0.5496721615791321, metrics={'train_runtime': 33878.711, 'train_samples_per_second': 0.708, 'train_steps_per_second': 0.089, 'total_flos': 7329322967040000.0, 'train_loss': 0.5496721615791321, 'epoch': 10.0})

In [17]:
    # Save the model
model.save_pretrained("./codeT5-correctness-gcj(Bin-exp2)")
tokenizer.save_pretrained("./codeT5-correctness-gcj(Bin-exp2)")


('./codeT5-correctness-gcj(Bin-exp2)/tokenizer_config.json',
 './codeT5-correctness-gcj(Bin-exp2)/special_tokens_map.json',
 './codeT5-correctness-gcj(Bin-exp2)/vocab.json',
 './codeT5-correctness-gcj(Bin-exp2)/merges.txt',
 './codeT5-correctness-gcj(Bin-exp2)/added_tokens.json',
 './codeT5-correctness-gcj(Bin-exp2)/tokenizer.json')

In [23]:
trainer.evaluate()

/home/profniggastein/PycharmProjects/ReducedDataset/.venv/lib/python3.11/site-packages/torch/utils/data/dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


ValueError: expected sequence of length 2 at dim 2 (got 256)